In [ ]:
import jax
import jax.numpy as jnp
from jax import jit, vmap, grad
from jax import random
import optax
import numpy as np
import pickle

In [ ]:
def init_network_params(sizes, key=random.PRNGKey(4)):
    def random_layer_params(m, n, key, scale=1e-2):
        w_key, b_key = random.split(key)
        return scale * random.uniform(w_key, (n, m), minval=1, maxval=3), scale * random.uniform(b_key, (n,), minval=1, maxval=3)

    keys = random.split(key, len(sizes))
    return [random_layer_params(m, n, k) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

def predict(NN, u):
    activations = u.reshape((-1, ))
    activations = jnp.log(activations/(1-activations))
    for w, b in NN[:-1]:
        outputs = jnp.dot(w, activations) + b
        activations = jax.nn.sigmoid(outputs)
    final_w, final_b = NN[-1]
    logit = jnp.dot(final_w, activations) + final_b
    return logit.squeeze()

vpredict = vmap(predict, in_axes=(None, 0))
gpredict = grad(predict, argnums=1)
vgpredict = jit(vmap(gpredict, in_axes=(None, 0)))

In [ ]:
def pi(x):
    return (1 / jnp.sqrt(2 * jnp.pi)) * jnp.exp(-0.5 * x**2)
vpi = vmap(pi, in_axes=(0,))

@jit
def obj(NN, u_batch):
    def ell(NN, u):
        return -jnp.log(pi(predict(NN, u))) - jnp.log(gpredict(NN, u))
    vell = vmap(ell, in_axes=(None, 0))

    result = vell(NN, u_batch)
    result = jnp.mean(result)
    return result

@jit
def constraint(NN, u_batch):
    epsilon = 1e-12
    def g(NN, u):
        return jnp.maximum(-gpredict(NN, u)+epsilon, 0)
    vg = vmap(g, in_axes=(None, 0))

    result = vg(NN, u_batch)
    result = jnp.sum(result)
    return result

gobj = jit(grad(obj))
gconst = jit(grad(constraint))

@jit
def evaluation(NN, test_uz):
    u_batch, z_batch = test_uz[:, 0], test_uz[:, 1]
    result = jnp.square(vpredict(NN, u_batch) - z_batch)
    result = jnp.mean(result)
    return result

In [25]:
LAYER_SIZES = [1, 10, 10, 10, 1]
NN = init_network_params(LAYER_SIZES, random.PRNGKey(15))

optimizer= optax.adam(0.001)
opt_state = optimizer.init(NN)
bug = 0
co = 0
test_uz = np.load(f'/content/drive/MyDrive/No. 22 Physica D (inverse)/Normal checkpoints (TM)/test_uz.npy')
lowest = evaluation(NN, test_uz)
Lloss = []
Llowest = []
NN_best = NN
for epoch in range(2000000):
    u_batch = np.random.uniform(0, 1, (10000, 1))
    while jnp.isnan(vgpredict(NN, u_batch)).sum() != 0:
        u_batch = np.random.uniform(0, 1, (10000, 1))
        bug += 1

    while constraint(NN, u_batch).item()>0:
        grads = gconst(NN, u_batch)
        updates, opt_state = optimizer.update(grads, opt_state)
        NN = optax.apply_updates(NN, updates)
        co += 1

    grads = gobj(NN, u_batch)
    updates, opt_state = optimizer.update(grads, opt_state)
    NN = optax.apply_updates(NN, updates)

    Lloss.append(obj(NN, u_batch).item())
    Llowest.append(lowest.item())

    if  evaluation(NN, test_uz)<lowest:
        lowest = evaluation(NN, test_uz)
        NN_best = NN

    if epoch % 10000 == 0:
        print(f"Epoch: {epoch}, loss:{obj(NN, u_batch).item():.7f}, const:{constraint(NN, u_batch).item():.4f}, lowest:{lowest:.4f}, co:{co}, bug:{bug}")
        np.save(f'/content/drive/MyDrive/No. 22 Physica D (inverse)/Normal checkpoints (TM)/Lloss.npy', Lloss)
        np.save(f'/content/drive/MyDrive/No. 22 Physica D (inverse)/Normal checkpoints (TM)/Llowest.npy', Llowest)
        with open(f'/content/drive/MyDrive/No. 22 Physica D (inverse)/Normal checkpoints (TM)/NN (best).pkl', 'wb') as f:
            pickle.dump(NN_best, f)
        with open(f'/content/drive/MyDrive/No. 22 Physica D (inverse)/Normal checkpoints (TM)/NN_{epoch}.pkl', 'wb') as f:
            pickle.dump(NN, f)

Epoch: 0, loss:11.5279713, const:0.0000, lowest:4.7293, co:0, bug:0
Epoch: 10000, loss:0.0019044, const:0.0000, lowest:0.0271, co:0, bug:1
Epoch: 20000, loss:0.0006334, const:0.0000, lowest:0.0061, co:0, bug:2
Epoch: 30000, loss:0.0002826, const:0.0000, lowest:0.0006, co:0, bug:2
Epoch: 40000, loss:0.0000043, const:0.0000, lowest:0.0004, co:0, bug:5
Epoch: 50000, loss:0.0000858, const:0.0000, lowest:0.0003, co:0, bug:8
Epoch: 60000, loss:0.0001902, const:0.0000, lowest:0.0002, co:0, bug:11
Epoch: 70000, loss:0.0000482, const:0.0000, lowest:0.0002, co:0, bug:15
Epoch: 80000, loss:0.0000169, const:0.0000, lowest:0.0002, co:0, bug:17
Epoch: 90000, loss:0.0002682, const:0.0000, lowest:0.0001, co:0, bug:24
Epoch: 100000, loss:-0.0000706, const:0.0000, lowest:0.0001, co:0, bug:28
Epoch: 110000, loss:-0.0000089, const:0.0000, lowest:0.0001, co:0, bug:30
Epoch: 120000, loss:0.0001181, const:0.0000, lowest:0.0001, co:0, bug:32
Epoch: 130000, loss:0.0001798, const:0.0000, lowest:0.0001, co:0, bu